You can use this exercise as a guide to predict house prices using numerical in a real dataset. It is based on this tutorial https://www.pyimagesearch.com/2019/01/21/regression-with-keras/.

In [ ]:
# Downloading the data
!git clone https://github.com/emanhamed/Houses-dataset
!ls -la Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 37.70 MiB/s, done.
Resolving deltas: 100% (20/20), done.
total 100
drwxr-xr-x 4 root root  4096 Jan 14 20:34  .
drwxr-xr-x 1 root root  4096 Jan 14 20:33  ..
drwxr-xr-x 8 root root  4096 Jan 14 20:34  .git
-rw-r--r-- 1 root root   378 Jan 14 20:34  .gitattributes
-rw-r--r-- 1 root root  1726 Jan 14 20:34  .gitignore
drwxr-xr-x 2 root root 73728 Jan 14 20:34 'Houses Dataset'
-rw-r--r-- 1 root root  1767 Jan 14 20:34  README.md


In [ ]:
# Showing some lines in the txt file
!head -n 10 Houses-dataset/Houses\ Dataset/HousesInfo.txt 

4 4 4053 85255 869500
4 3 3343 36372 865200
3 4 3923 85266 889000
5 5 4022 85262 910000
3 4 4116 85266 971226
4 5 4581 85266 1249000
3 4 2544 85262 799000
4 5 5524 85266 1698000
3 4 4229 85255 1749000
4 5 3550 85262 1500000


In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

def load_house_attributes(inputPath):

	# initialize the list of column names in the CSV file and then load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
 
  # determine (1) the unique zip codes and (2) the number of data points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
 
	# loop over each of the unique zip codes and their corresponding count

	for (zipcode, count) in zip(zipcodes, counts):
   
		# the zip code counts for our housing dataset is *extremely* unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
   
	# return the data frame
	return df

def process_house_attributes(df, train, test):

	# initialize the column names of the continuous data
	continuous = ["bedrooms", "bathrooms", "area"]
  
	# performin min-max scaling each continuous feature column to the range [0, 1]
	cs = MinMaxScaler()
	trainContinuous = cs.fit_transform(train[continuous])
	testContinuous = cs.transform(test[continuous])
 
  # one-hot encode the zip code categorical data (by definition of one-hot encoing, all output features are now in the range [0, 1])
	zipBinarizer = LabelBinarizer().fit(df["zipcode"])
	trainCategorical = zipBinarizer.transform(train["zipcode"])
	testCategorical = zipBinarizer.transform(test["zipcode"])
 
	# construct our training and testing data points by concatenating the categorical features with the continuous features
	trainX = np.hstack([trainCategorical, trainContinuous])
	testX = np.hstack([testCategorical, testContinuous])
 
	# return the concatenated training and testing data
	return (trainX, testX)

Let's define the model as in the tutorial:

<img src="https://www.pyimagesearch.com/wp-content/uploads/2019/01/keras_regression_arch.png">

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_mlp(dim, regress=False):

	# define our MLP network
	model = Sequential()
	model.add(Dense(16, input_dim=dim, activation="relu"))
	model.add(Dense(8, activation="relu"))
	model.add(Dense(4, activation="relu"))
 
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
  
	# return our model
	return model

In [ ]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import locale
import os

# construct the path to the input .txt file that contains information on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")

# inputPath = os.path.sep.join([args["dataset"], "HousesInfo.txt"])
inputPath = "Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath)

# construct a training and testing split with 75% of the data used for training and the remaining 25% for evaluation
print("[INFO] constructing training/testing split...")
(train, test) = train_test_split(df, test_size=0.25, random_state=42)

[INFO] loading house attributes...
[INFO] constructing training/testing split...


In [ ]:
print(train["price"].max())
print(train["price"].min())

5858000
36000


In [ ]:
# find the largest house price in the training set and use it to scale our house prices to the range [0, 1] (this will lead to better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [ ]:
print(trainY.max())
print(trainY.min())

1.0
0.006145442130419939


In [ ]:
# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
print("[INFO] processing data...")
(trainX, testX) = process_house_attributes(df, train, test)

[INFO] processing data...


In [ ]:
print(trainX.shape)
print(testX.shape)

(271, 10)
(91, 10)


In [ ]:
# create our MLP and then compile the model using mean absolute percentage error as our loss, implying that we seek to minimize
# the absolute percentage difference between our price *predictions* and the *actual prices*

model = create_mlp(trainX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# train the model
print("[INFO] training model...")
model.fit(x=trainX, y=trainY, 
	        validation_data=(testX, testY),
	        epochs=200, batch_size=8)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] training model...
Epoch 1/200
34/34 [==============================] - 1s 6ms/step - loss: 104.4267 - val_loss: 82.5521
Epoch 2/200
34/34 [==============================] - 0s 3ms/step - loss: 60.2591 - val_loss: 48.1079
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 42.0330 - val_loss: 31.2145
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 31.0133 - val_loss: 28.7841
Epoch 5/200
34/34 [==============================] - 0s 2ms/step - loss: 28.8872 - val_loss: 28.3920
Epoch 6/200
34/34 [==============================] - 0s 3ms/step - loss: 30.0653 - val_loss: 27.9217
Epoch 7/200
34/34 [==============================] - 0s 3ms/step - loss: 28.3277 - val_loss: 29.8467
Epoch 8/200
34/34 [==============================] - 0s 3ms/step - loss: 32.4921 - val_loss: 30.7374
Epoch 9/200
34/34 [==============================] - 0s 3ms/step - loss: 32.2018 - val_loss: 25.8013
Epoch 10/200
34/34 [==============================] - 0s 3ms/step

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 27.84%, std: 27.32%


Here it is the example that combines numerical data and images https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/.
